# Loading Data

In [1]:
import tensorflow as tf

# Loading MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
working_dir = '/Users/ndjido/Documents/Davidson_Consulting/TF_DL_Meetup/Demo/Deep_MNIST/'

### Summary function

In [3]:
def variable_summaries(var, name):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev/' + name , stddev)
        tf.summary.scalar('max/' + name, tf.reduce_max(var))
        tf.summary.scalar('min/' + name, tf.reduce_min(var))
        tf.summary.histogram('histogram/' + name, var)

###  Network Building-block Helpers

In [4]:
def create_layer(inputs, inputs_dim, output_dim, name, softmax = False, p_dropout = 0):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal([inputs_dim, output_dim], stddev=0.1), name="W")
        variable_summaries(W, 'Weights')
        b = tf.Variable(tf.truncated_normal([output_dim]), name="b")
        variable_summaries(b, 'Biases')
        layer = tf.nn.relu(tf.matmul(inputs, W) + b)
        if softmax == False:
            if p_dropout > 0 and p < 1:
                return tf.nn.dropout(layer, p_dropout)
            else:
                return layer
        else:
            return tf.nn.softmax(layer)

## Building the Network

In [5]:
# Hidden Layers Dimensions
A = 200
B = 100
C = 60
D = 30

# Output Layer
_output_dim = 10

X = tf.placeholder(tf.float32, shape=[None, 28 * 28])
Y_ = tf.placeholder(tf.float32, shape=[None, _output_dim])   

Y1 = create_layer(X, 28 * 28, A, "layer_A")
Y2 = create_layer(Y1, A, B, "layer_B")
Y3 = create_layer(Y2, B, C, "layer_C")
Y4 = create_layer(Y3, C, D, "layer_D")
Y = create_layer(Y4, D, _output_dim, "Fully_connected", True, .7)

# Loss Function (or Cost Function)
with tf.name_scope('cross_entropy'):
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=Y_))

tf.summary.scalar("cross_entropy_loss", cross_entropy_loss)

<tf.Tensor 'cross_entropy_loss:0' shape=() dtype=string>

## Training operation

In [6]:
with tf.name_scope("Trainer"):
    trainer = tf.train.GradientDescentOptimizer(.5).minimize(cross_entropy_loss)

## Accuracy operation

In [7]:
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

## Running the training of the model

In [ ]:
with tf.Session() as sess:
    #Training
        
    sess.run(tf.global_variables_initializer())
    
    # Summary writer for TensorBoad  
    merged_summary = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(working_dir + 'train', sess.graph)
    test_writer = tf.summary.FileWriter(working_dir + 'test')
    
    for i in range(10000):
        # Train
        x_data, y_data = mnist.train.next_batch(100)
        summary, _ = sess.run([merged_summary, trainer], feed_dict={X: x_data, Y_: y_data})
        train_writer.add_summary(summary, i)
        # Test
        summary, acc = sess.run([merged_summary, accuracy], feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
        test_writer.add_summary(summary, i)
        print('Step %s -> Accuracy %s' % (i, acc))

Step 0 -> Accuracy 0.096
Step 1 -> Accuracy 0.0821
Step 2 -> Accuracy 0.096
Step 3 -> Accuracy 0.0973
Step 4 -> Accuracy 0.1086
Step 5 -> Accuracy 0.1288
Step 6 -> Accuracy 0.1443
Step 7 -> Accuracy 0.1576
Step 8 -> Accuracy 0.1594
Step 9 -> Accuracy 0.1078
Step 10 -> Accuracy 0.1646
Step 11 -> Accuracy 0.1654
Step 12 -> Accuracy 0.1503
Step 13 -> Accuracy 0.1568
Step 14 -> Accuracy 0.0995
Step 15 -> Accuracy 0.0958
Step 16 -> Accuracy 0.0958
Step 17 -> Accuracy 0.0958
Step 18 -> Accuracy 0.0958
Step 19 -> Accuracy 0.0958
Step 20 -> Accuracy 0.1087
Step 21 -> Accuracy 0.1288
Step 22 -> Accuracy 0.1575
Step 23 -> Accuracy 0.143
Step 24 -> Accuracy 0.1593
Step 25 -> Accuracy 0.0958
Step 26 -> Accuracy 0.2022
Step 27 -> Accuracy 0.1797
Step 28 -> Accuracy 0.1568
Step 29 -> Accuracy 0.1928
Step 30 -> Accuracy 0.2111
Step 31 -> Accuracy 0.1412
Step 32 -> Accuracy 0.2689
Step 33 -> Accuracy 0.1816
Step 34 -> Accuracy 0.1365
Step 35 -> Accuracy 0.1693
Step 36 -> Accuracy 0.1122
Step 37 -> Acc

## Accuracy = 97.3%